In [150]:
%load_ext autoreload
%autoreload 2
from utils import*
from preprocessing import preprocessing
from feature_extraction import extract_features,read_images
from training import *
from performance import performance_analysis


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Preprocessing

In [14]:
#Training Data Set
for i in range(0, 6):
    print(i)
    path_of_the_directory='../data_simple/split/men/train/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/men/train/'+str(i)+'/'
    isExist = os.path.exists(path_of_the_directory_result)
    if not isExist:
        os.makedirs(path_of_the_directory_result)
        print("The new directory is created!")
    else:
        shutil.rmtree(path_of_the_directory_result, ignore_errors=False, onerror=None)
        os.mkdir(path_of_the_directory_result)
    # ind=0
    for filename in os.listdir(path_of_the_directory):
        # if(ind>20):
        #     break
        # ind=ind+1
        img = cv2.imread(path_of_the_directory+str(filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preprocessing(img, gamma=True, close=True)
        cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

    path_of_the_directory='../data_simple/split/women/train/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/women/train/'+str(i)+'/'
    isExist = os.path.exists(path_of_the_directory_result)
    if not isExist:
        os.makedirs(path_of_the_directory_result)
        print("The new directory is created!")
    # ind=0
    for filename in os.listdir(path_of_the_directory):
        # if(ind>20):
        #     break
        # ind=ind+1
        img = cv2.imread(path_of_the_directory+str( filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preprocessing(img, gamma=True, close=True)
        cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

0
1
The new directory is created!
2
The new directory is created!
The new directory is created!
3
The new directory is created!
The new directory is created!
4
The new directory is created!
The new directory is created!
5
The new directory is created!
The new directory is created!


In [16]:
#Testing Data Set
for i in range(0, 6):
    print(i)
    path_of_the_directory='../data_simple/split/men/test/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/men/test/'+str(i)+'/'
    isExist = os.path.exists(path_of_the_directory_result)
    if not isExist:
        os.makedirs(path_of_the_directory_result)
        print("The new directory is created!")
    # ind=0
    for filename in os.listdir(path_of_the_directory):
        # if(ind>20):
        #     break
        # ind=ind+1
        img = cv2.imread(path_of_the_directory+str(filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preprocessing(img, gamma=True, close=True)
        cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

    path_of_the_directory='../data_simple/split/women/test/'+str(i)+'/'
    path_of_the_directory_result='../outputs/preprocessing_output/women/test/'+str(i)+'/'
    isExist = os.path.exists(path_of_the_directory_result)
    if not isExist:
        os.makedirs(path_of_the_directory_result)
        print("The new directory is created!")
    # ind=0
    for filename in os.listdir(path_of_the_directory):
        # if(ind>20):
        #     break
        # ind=ind+1
        img = cv2.imread(path_of_the_directory+str( filename))
        if img is None:
            continue

        # Skin Masks
        binary_hand=preprocessing(img, gamma=True, close=True)
        cv2.imwrite(path_of_the_directory_result+filename,binary_hand)

0
The new directory is created!
The new directory is created!
1
The new directory is created!
The new directory is created!
2
The new directory is created!
The new directory is created!
3
The new directory is created!
The new directory is created!
4
The new directory is created!
The new directory is created!
5
The new directory is created!
The new directory is created!


Feature Extraction

In [67]:
# Step(1) Read Images
path='../outputs/preprocessing_output/'
train_images=read_images(path,type="train")

print("Read 0:", np.shape(train_images['0']))
print("Read 1:", np.shape(train_images['1']))
print("Read 2:", np.shape(train_images['2']))
print("Read 3:", np.shape(train_images['3']))
print("Read 4:", np.shape(train_images['4']))
print("Read 5:", np.shape(train_images['5']))


test_images=read_images(path,type="test")

print("Read 0:", np.shape(test_images['0']))
print("Read 1:", np.shape(test_images['1']))
print("Read 2:", np.shape(test_images['2']))
print("Read 3:", np.shape(test_images['3']))
print("Read 4:", np.shape(test_images['4']))
print("Read 5:", np.shape(test_images['5']))



0
1
2
3
4
5
Read 0: (2, 2592, 4608)
Read 1: (3, 2592, 4608)
Read 2: (3, 2592, 4608)
Read 3: (4, 2592, 4608)
Read 4: (3, 2592, 4608)
Read 5: (3, 2592, 4608)
0
1
2
3
4
5
Read 0: (2, 2592, 4608)
Read 1: (2, 2592, 4608)
Read 2: (2, 2592, 4608)
Read 3: (2, 2592, 4608)
Read 4: (2, 2592, 4608)
Read 5: (2, 2592, 4608)


In [141]:
number_of_clusters=8
path='../outputs/preprocessing_output/'
X_train,Y_train,visual_words=extract_features(path,number_of_clusters,debug=True,images=train_images,train=True,visual_words=None)
# print(np.shape(visual_words)



sift_descriptors for 0
0 has: 492Keypoints
1 has: 597Keypoints
sift_descriptors for 1
0 has: 308Keypoints
1 has: 534Keypoints
2 has: 754Keypoints
sift_descriptors for 2
0 has: 909Keypoints
1 has: 736Keypoints
2 has: 496Keypoints
sift_descriptors for 3
0 has: 1398Keypoints
1 has: 338Keypoints
2 has: 1112Keypoints
3 has: 519Keypoints
sift_descriptors for 4
0 has: 883Keypoints
1 has: 965Keypoints
2 has: 1097Keypoints
sift_descriptors for 5
0 has: 753Keypoints
1 has: 1043Keypoints
2 has: 625Keypoints


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


0 (2,)
1 (3,)
2 (3,)
3 (4,)
4 (3,)
5 (3,)


In [146]:
# creating df object with columns specified   
csv = pd.DataFrame(X_train, columns =list(range(number_of_clusters))) 

# #Adding Lables to the Data Frame
csv.insert(0, "class", Y_train)
# print(csv)


csv.to_csv (r'../outputs/train.csv', index = None)

   class      0      1      2      3      4      5      6      7
0      0   53.0  216.0   16.0   18.0   78.0   35.0   20.0   56.0
1      0   89.0  136.0   46.0   62.0   92.0   59.0   36.0   77.0
2      1   60.0   38.0   17.0   24.0   47.0   33.0   15.0   74.0
3      1   70.0  213.0   26.0   26.0   60.0   46.0   27.0   66.0
4      1  132.0  214.0   56.0   60.0   66.0   78.0   57.0   91.0
5      2  162.0  160.0   80.0   93.0   89.0   92.0   88.0  145.0
6      2  136.0  117.0   56.0   54.0   75.0   92.0   56.0  150.0
7      2   62.0  213.0   24.0   22.0   49.0   43.0   21.0   62.0
8      3  280.0  269.0  160.0  167.0   94.0  112.0  170.0  146.0
9      3   56.0   66.0   36.0   27.0   45.0   31.0   32.0   45.0
10     3  204.0  213.0  126.0  157.0   99.0   58.0  150.0  105.0
11     3   97.0   68.0   68.0   66.0   30.0   44.0   63.0   83.0
12     4  151.0  235.0   67.0   88.0   80.0   74.0   84.0  104.0
13     4  190.0  116.0  121.0  146.0   88.0   76.0  113.0  115.0
14     4  184.0  251.0   

In [147]:
path='../outputs/preprocessing_output/'
X_test,Y_test,visual_words=extract_features(path,clusters=None,debug=True,images=test_images,train=False,visual_words=visual_words)

sift_descriptors for 0
0 has: 616Keypoints
1 has: 480Keypoints
sift_descriptors for 1
0 has: 621Keypoints
1 has: 74Keypoints
sift_descriptors for 2
0 has: 680Keypoints
1 has: 591Keypoints
sift_descriptors for 3
0 has: 771Keypoints
1 has: 981Keypoints
sift_descriptors for 4
0 has: 750Keypoints
1 has: 798Keypoints
sift_descriptors for 5
0 has: 596Keypoints
1 has: 1004Keypoints
0 (2,)
1 (2,)
2 (2,)
3 (2,)
4 (2,)
5 (2,)


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [148]:
# creating df object with columns specified   
csv = pd.DataFrame(X_test, columns =list(range(number_of_clusters))) 

#Adding Lables to the Data Frame

csv.insert(0, "class", Y_test)
print(csv)


csv.to_csv (r'../outputs/test.csv', index = None)

   class      0      1     2     3      4      5     6      7
0      0   88.0  200.0  42.0  40.0   73.0   58.0  37.0   78.0
1      0   56.0  189.0  19.0  21.0   63.0   44.0  14.0   74.0
2      1   84.0  216.0  37.0  50.0   66.0   44.0  42.0   82.0
3      1   13.0   11.0   9.0  11.0   11.0    4.0   9.0    6.0
4      2   61.0  406.0  29.0  28.0   43.0   32.0  19.0   62.0
5      2   62.0  294.0  11.0  14.0   54.0   53.0  14.0   89.0
6      3   92.0  380.0  41.0  41.0   46.0   42.0  52.0   77.0
7      3  150.0  310.0  79.0  88.0   79.0   85.0  68.0  122.0
8      4   77.0  356.0  19.0  19.0   88.0   72.0  14.0  105.0
9      4  121.0  253.0  49.0  62.0   96.0   79.0  49.0   89.0
10     5   65.0  316.0  20.0  18.0   52.0   45.0  22.0   58.0
11     5  137.0  366.0  46.0  46.0  128.0  114.0  34.0  133.0


Training

In [ ]:
# Read from CVS File
df = pd.read_csv('../outputs/features.csv')
# print(df)


classes=df.loc[:,"class"]
classes = classes.to_numpy()
# print(classes)
# print(np.shape(classes))


features=df.loc[:, df.columns != 'class']
features = features.to_numpy()
# print(features)
# print(np.shape(features))

In [157]:
prediction, actual = train_svm(X_train,Y_train,X_test,Y_test)
# prediction, actual = train_randomforest(X_train,Y_train,X_test,Y_test)

# print(prediction)
# print(actual)

Accuracy

In [158]:
accuracy = performance_analysis(prediction,actual)
print(accuracy)

0.16666666666666666
